In [64]:
import pandas as pd
import os, sys
from collections import Counter

In [71]:
class Match:
    def __init__(self, data):
        self.parse_data(data)
        
    def parse_data(self, data):
        self.match_id     = data.iloc[0, 0]
        self.date         = data.iloc[0, 1]
        self.win_side     = data.iloc[0, 2]
        self.lose_side    = data.iloc[0, 4]
        self.win_players  = data.iloc[1:6, 2].values
        self.lose_players = data.iloc[1:6, 4].values
        self.picks = {}
        for i, x in enumerate(data.iloc[1:6, 2]):
            self.picks[x] = data.iloc[(1+i), 3]
        for i, x in enumerate(data.iloc[1:6, 4]):
            self.picks[x] = data.iloc[(1+i), 5]
    
    def __str__(self):
        tostr = ""
        tostr += ("Match ID: {}\n".format(self.match_id))
        tostr += ("Date: {}\n".format(self.date))
        tostr += ("Win Side: {}\n".format(self.win_side))
        tostr += ("Lose Side: {}\n".format(self.lose_side))
        tostr += ("Win Players: {}\n".format(self.win_players))
        tostr += ("Lose Players: {}\n".format(self.lose_players))
        tostr += ("Picks: {}".format(self.picks))
        return tostr

def parse_google_docs_sheet(fn):
    x = pd.read_csv(fn)
    assert x.shape[0] % 7 == 6 # one less than a multiple of 7
    x = x.iloc[:, 0:6]

    match_block_height = 6
    idx = 0
    matches = []
    while idx < x.shape[0]:
        block = x.iloc[idx:(idx+match_block_height), :]
        matches.append(Match(block))
        idx += (match_block_height+1)    
    return matches

def calc_win_pct(matches):
    wl_dict = {}
    for match in matches:
        for wp in match.win_players:
            if wp in wl_dict:
                wl_dict[wp][0] += 1
            else:
                wl_dict[wp] = [1, 0]
                
        for lp in match.lose_players:
            if lp in wl_dict:
                wl_dict[lp][1] += 1
            else:
                wl_dict[lp] = [0, 1]
    df = pd.DataFrame(wl_dict).T
    df.columns = ['W', 'L']
    return df

def calc_heroes_played(matches):
    hp_dict = {}
    for match in matches:
        for pick in match.picks:
            if pick in hp_dict:
                hp_dict[pick].append(match.picks[pick])
            else:
                hp_dict[pick] = [match.picks[pick]]
    for x in hp_dict:
        hp_dict[x] = Counter(hp_dict[x])
    return hp_dict

In [66]:
matches = parse_google_docs_sheet('Inhouse stats - 2020 - Sheet1.csv')

In [67]:
for match in matches:
    print('-----------')
    print(match)

-----------
Match ID: Match 1
Date: 1/11/2020
Win Side: Radiant
Lose Side: Dire
Win Players: ['Bibop0p - Jordan' 'coolcrimefighter - Cody' 'BlackWater - Greg'
 'hunger - Nate' 'Kent8vious C24ldwell-Pope - Brandon']
Lose Players: ['pigu - Sam' 'OKC won 0 titles with 3 MVPs - Albert'
 'kimchichouder - Kim' '（=´∇｀=） - Jason' 'Dorkhead - Michael']
Picks: {'pigu - Sam': 'Treant', 'Bibop0p - Jordan': 'Slardar', 'hunger - Nate': 'Crystal Maiden', 'BlackWater - Greg': 'Viper', 'Dorkhead - Michael': 'Vengeful Spirit', 'coolcrimefighter - Cody': 'Dark Seer', '（=´∇｀=） - Jason': 'Magnus', 'kimchichouder - Kim': 'Abaddon', 'Kent8vious C24ldwell-Pope - Brandon': 'Lifestealer', 'OKC won 0 titles with 3 MVPs - Albert': 'Storm Spirit'}
-----------
Match ID: Match 2
Date: 1/14/2020
Win Side: Dire
Lose Side: Radiant
Win Players: ['BlackWater - Greg' 'Bibop0p - Jordan' 'SlowDrawing - Tony'
 'coolcrimefighter - Cody' 'Dorkhead - Michael']
Lose Players: ['kimchichouder - Kim' 'hunger - Nate'
 'Kent8vious C2

In [68]:
calc_win_pct(matches)

,W,L
2mg - Jimmy,1,0
Bibop0p - Jordan,4,4
BlackWater - Greg,5,2
Dorkhead - Michael,5,3
Kent8vious C24ldwell-Pope - Brandon,3,6
OKC won 0 titles with 3 MVPs - Albert,4,5
Pandamonium Express - Ryan,3,0
SlowDrawing - Tony,3,1
coolcrimefighter - Cody,4,5
hoodie alan - Alan,1,3


In [83]:
hp_counter = calc_heroes_played(matches)
for x in hp_counter:
    print(x)
    most_common = hp_counter[x].most_common(1)
    print(most_common[0][0], most_common[0][1])
    print('--')

hunger - Nate
Disruptor 2
--
BlackWater - Greg
Winter Wyvern 2
--
coolcrimefighter - Cody
Dark Seer 2
--
SlowDrawing - Tony
Puck 1
--
hoodie alan - Alan
Mars 1
--
Kent8vious C24ldwell-Pope - Brandon
Lifestealer 3
--
kimchichouder - Kim
Winter Wyvern 2
--
2mg - Jimmy
Invoker 1
--
pigu - Sam
Monkey King 2
--
Bibop0p - Jordan
Slardar 2
--
Pandamonium Express - Ryan
Undying 1
--
（=´∇｀=） - Jason
Razor 1
--
Dorkhead - Michael
Lich 2
--
OKC won 0 titles with 3 MVPs - Albert
Storm Spirit 1
--


In [99]:
# Versatility
for x in hp_counter:
    print("{}: {}/{}".format(x, len(hp_counter[x].keys()), sum(hp_counter[x].values())))

hunger - Nate: 8/9
BlackWater - Greg: 6/7
coolcrimefighter - Cody: 8/9
SlowDrawing - Tony: 4/4
hoodie alan - Alan: 4/4
Kent8vious C24ldwell-Pope - Brandon: 6/9
kimchichouder - Kim: 5/6
2mg - Jimmy: 1/1
pigu - Sam: 7/8
Bibop0p - Jordan: 7/8
Pandamonium Express - Ryan: 3/3
（=´∇｀=） - Jason: 5/5
Dorkhead - Michael: 7/8
OKC won 0 titles with 3 MVPs - Albert: 9/9
